# A model for classifying various hair types.

In [ ]:
pip install --upgrade torch torchvision torchaudio

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
print(torch.__version__)



2.3.0


In [2]:
# Reproducibility settings
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
class baseCNN(nn.Module):
    def __init__(self):
        super(baseCNN, self).__init__()
        
        # Conv Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Flatten
        self.flatten = nn.Flatten()
        
        # The input size to the linear layer calculation:
        # Input: 200x200
        # Conv2d (3x3, s=1, p=0) -> (200 - 3)/1 + 1 = 198. Output: 32 x 198 x 198
        # MaxPool2d (2x2) -> 198 / 2 = 99. Output: 32 x 99 x 99
        # Flatten size = 32 * 99 * 99 = 313,632
        
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x # We return logits, activation handled by Loss function/Training loop

The problem is Binary Classification (1 output neuron).

CrossEntropyLoss is typically for Multi-class classification.

MSELoss is for regression.

The provided training loop code uses torch.sigmoid(outputs) to calculate accuracy. This implies the model outputs "logits" (raw scores) rather than probabilities. The most numerically stable loss function that takes logits and applies Sigmoid internally is BCEWithLogitsLoss.

*The total number of parameters of the model*
Answer: 20073473

* Calculation:

* Conv2d: (3 * 3 * 3 + 1) * 32 = 28 * 32 = 896 parameters. (kernel_h * kernel_w * input_channels + bias) * output_channels

* Linear 1: Input size is 32 * 99 * 99 = 313,632. 

* Weights: 313,632 * 64 = 20,072,448. Bias: 64. Total: 20,072,512.

* Linear 2: 64 * 1 (weights) + 1 (bias) = 65.

* Total: 896 + 20,072,512 + 65 = 20,073,473

## Training 

In [4]:
# 1. Transformations
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [9]:
# 2. Data Loaders (Assuming data is in './data')
train_dataset = datasets.ImageFolder('data/train', transform=train_transforms)
test_dataset = datasets.ImageFolder('data/test', transform=test_transforms)

# Important: Shuffle=True for train, False for test
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [11]:
# 3. Model Initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = baseCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [12]:
# 4. Training Loop
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) 

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

# --- ANSWERS CALCULATION ---
print("--- Results ---")
print(f"Median Training Accuracy: {np.median(history['acc'])}")
print(f"Std Dev Training Loss: {np.std(history['loss'])}")

Epoch 1/10 - Loss: 0.6492, Acc: 0.6350
Epoch 2/10 - Loss: 0.5565, Acc: 0.7050
Epoch 3/10 - Loss: 0.5461, Acc: 0.7200
Epoch 4/10 - Loss: 0.4670, Acc: 0.7675
Epoch 5/10 - Loss: 0.4307, Acc: 0.7937
Epoch 6/10 - Loss: 0.3459, Acc: 0.8438
Epoch 7/10 - Loss: 0.2653, Acc: 0.8950
Epoch 8/10 - Loss: 0.2238, Acc: 0.9175
Epoch 9/10 - Loss: 0.1496, Acc: 0.9413
Epoch 10/10 - Loss: 0.1669, Acc: 0.9337
--- Results ---
Median Training Accuracy: 0.81875
Std Dev Training Loss: 0.1668939442085757


In [13]:
# 1. Define Augmentations
train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)), # Ensure size is back to 200 if crop changed it oddly, though RandomResizedCrop handles it
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
# 2. Reload Training Data with Augmentations
train_dataset_aug = datasets.ImageFolder('data/train', transform=train_transforms_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)

# Note: We do NOT re-initialize the model. We continue training.

In [16]:
# 3. Training Loop (epochs 11-20)
num_epochs_aug = 10
history_aug = {'val_loss': [], 'val_acc': []}

print("Starting Augmentation Training...")

for epoch in range(num_epochs_aug):
    model.train()
    for images, labels in train_loader_aug: # Use new loader
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)
    
    print(f"Aug Epoch {epoch+1}/{num_epochs_aug} - Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# --- ANSWERS CALCULATION ---
print("--- Augmentation Results ---")
print(f"Mean Test Loss (All Aug Epochs): {np.mean(history_aug['val_loss'])}")
print(f"Mean Test Acc (Last 5 Epochs): {np.mean(history_aug['val_acc'][5:])}")

Starting Augmentation Training...
Aug Epoch 1/10 - Val Loss: 0.5951, Val Acc: 0.6915
Aug Epoch 2/10 - Val Loss: 0.6738, Val Acc: 0.6866
Aug Epoch 3/10 - Val Loss: 0.6871, Val Acc: 0.7114
Aug Epoch 4/10 - Val Loss: 0.5586, Val Acc: 0.7065
Aug Epoch 5/10 - Val Loss: 0.6166, Val Acc: 0.6667
Aug Epoch 6/10 - Val Loss: 0.6021, Val Acc: 0.7065
Aug Epoch 7/10 - Val Loss: 0.7201, Val Acc: 0.6318
Aug Epoch 8/10 - Val Loss: 0.8214, Val Acc: 0.6418
Aug Epoch 9/10 - Val Loss: 0.5524, Val Acc: 0.7363
Aug Epoch 10/10 - Val Loss: 0.5431, Val Acc: 0.7313
--- Augmentation Results ---
Mean Test Loss (All Aug Epochs): 0.6370412064873757
Mean Test Acc (Last 5 Epochs): 0.6895522388059703
